In [ ]:
#imports
import numpy as np
from at_synapse_detection import processDetections as pd
from at_synapse_detection import SynapseDetection as syn
from at_synapse_detection import dataAccess as da
from at_synapse_detection import synaptogram
import importlib
import matplotlib.pyplot as plt
from skimage import measure
import scipy.ndimage as ndimage
from shapely import geometry
import math

In [ ]:
importlib.reload(pd)
importlib.reload(syn)
importlib.reload(da)

In [ ]:
args = {
    "EM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json",
    "LM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/results/resultVol9.json",
    "EM_metadata_csv":"../data/M247514_Rorb_1/Site3Align2/MNSite3Synaptograms_v2.csv",
    "LM_metadata_file":"../data/M247514_Rorb_1/Site3Align2/site3_metadata.json",
    "EM_inclass_column":"glutsynapse",
    "EM_not_synapse_column":"ConsensusNotSynapse",
    "output_json":"../data/M247514_Rorb_1/Site3Align2/results/Anish_evaluation_output.json", 
    "annotationToRemove": "../data/M247514_Rorb_1/Site3Align2/missedanno.json"
    }

In [ ]:
metadataFN = '../data/M247514_Rorb_1/Site3Align2/site3_metadata.json'
metadata = syn.loadMetadata(metadataFN)
queryFN = metadata['querylocation']
listOfQueries = syn.loadQueriesJSON(queryFN)

In [ ]:
listOfQueryNumbers = [  0,   1,   2,   3,   4,   5,   6,   7]
listOfThresholds   = [0.8, 0.7, 0.7, 0.8, 0.7, 0.7, 0.8, 0.8]
queryresult = pd.combineResultVolumes(listOfQueryNumbers, listOfThresholds, metadata, args)

In [ ]:
good_annotations = queryresult['good_annotations']

In [ ]:
anno = good_annotations[151]
print(anno['oid'])
win_xy = 12
win_z = 2
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/'
query = listOfQueries[0]

In [ ]:
thresh = 0.7


In [ ]:
def checkQueryAgainstAnno(anno, query, thresh, win_xy, win_z, filepath): 
    
    bbox = synaptogram.getAnnotationBoundingBox2(anno)
    bbox = synaptogram.transformSynapseCoordinates(bbox)

    expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)
    zrange = list(range(expandedBox['startZ'], expandedBox['endZ']+1))

    synapticVolumes = loadSynapseDataFromQuery(query, anno, win_xy, win_z, filepath)
    resultVol = syn.getSynapseDetections(synapticVolumes, query)

    #print(thresh)
    SE = np.ones((2, 2, 2))
    dilated_volume = ndimage.binary_dilation(resultVol > thresh, SE) 
    
    synapsevol = np.zeros(resultVol.shape)

    synapseOutlinesDict = synaptogram.getAnnotationOutlines(anno)
    synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)
    listOfZinds = synapseOutlinesDict['zInds']
    listOfZinds = sorted(listOfZinds)
    
    startX = expandedBox['startX']
    startY = expandedBox['startY']
    minX = math.floor(bbox['minX'] - startX)
    maxX = math.ceil(bbox['maxX'] - startX)
    minY = math.floor(bbox['minY'] - startY)
    maxY = math.ceil(bbox['maxY'] - startY)

    offsetZ = 0 
    for localZ, globalZ in enumerate(zrange): 
        if zrange[localZ] == listOfZinds[offsetZ]: 
            synapsevol[minY:maxY, minX:maxX, localZ] = 1
            offsetZ = offsetZ + 1 

            if offsetZ == len(listOfZinds): 
                break


    foo = dilated_volume + synapsevol
    
    synapseDetected = False
    if foo.max() > 1: 
        print("Synapse Detected!")
        synapseDetected = True
    else: 
        print("Synapse not detected")
    return synapseDetected

In [ ]:
query

In [ ]:
n = 4 

In [ ]:
annoDetectedList = np.zeros(len(good_annotations), dtype=bool)

In [ ]:
for n, anno in enumerate(good_annotations): 
    print(anno['oid'])
    synapseDetected = checkQueryAgainstAnno(anno, query, thresh, win_xy, win_z, filepath)
    annoDetectedList[n] = synapseDetected

In [ ]:
len(annoDetectedList) - sum(annoDetectedList)

In [ ]:
%matplotlib notebook

plt.imshow(dilated_volume[:, :, n], cmap='gray')
plt.colorbar()
plt.plot(xcolumn, ycolumn, color='red', linewidth=2)


In [ ]:
def getVolume(bboxCoordinates, win_xy, win_z, volname, filepath): 
    """    
    Parameters
    -----------
    
    Returns
    -----------
    
    """
    
    # check for boundary issues
    startZ = bboxCoordinates['minZ']
    if (startZ - win_z > -1):
        startZ = startZ - win_z; 
    
    endZ = bboxCoordinates['maxZ']
    
    if (endZ + win_z < 50):
        endZ = endZ + win_z; 
    
    # get range of x, y values 
    startX = bboxCoordinates['minX'] - win_xy;
    startY = bboxCoordinates['minY'] - win_xy;
    deltaX = bboxCoordinates['maxX'] - startX + win_xy;
    deltaY = bboxCoordinates['maxY'] - startY + win_xy;
    
    startX = int(round(startX))
    startY = int(round(startY))
    deltaX = int(round(deltaX))
    deltaY = int(round(deltaY))
    startZ = int(round(startZ))
    endZ   = int(round(endZ))
    
    numSlices = endZ - startZ + 1
    
    # allocate volume
    vol = np.zeros((deltaY, deltaX, numSlices), dtype=np.float64)
        
    # iterate over each slice 
    sliceitr = 0 
    for sliceInd in range(startZ, endZ + 1):
      
        cutout = da.getImageCutoutFromFile(volname, sliceInd, startX, startY, deltaX, deltaY, filepath)
        cutout.astype(np.float64)
        cutout = syn.getProbMap(cutout)

        vol[:, :, sliceitr] = cutout; 
        sliceitr = sliceitr + 1 
        
    return vol

In [ ]:
def loadSynapseDataFromQuery(query, anno, win_xy, win_z, filepath):
    """
    Load tiff stacks associated with a query
    Parameters
    ----------
    query : dict - object containing filenames associated with pre/post synaptic markers 
    filepath : str - location of data 

    Returns
    ----------
    synapticVolumes : dict
        dict with two (pre/post) lists of synaptic volumes
    """

    bbox = synaptogram.getAnnotationBoundingBox2(anno)
    bbox = synaptogram.transformSynapseCoordinates(bbox)
    
    # query = {'preIF' : preIF, 'preIF_z' : preIF_z, 'postIF' : postIF, 'postIF_z' : postIF_z};

    #presynaptic volumes
    presynapticvolumes = []
    preIF = query['preIF']

    # Loop over every presynaptic channel 
    for n in range(0, len(preIF)):
        #print(preIF[n])
        
        volume = getVolume(bbox, win_xy, win_z, preIF[n], filepath)
        presynapticvolumes.append(volume)

    #postsynaptic volumes
    postsynapticvolumes = []
    postIF = query['postIF']

    # Loop over every postsynaptic channel 
    for n in range(0, len(postIF)):
       # print(postIF[n])
        volume = getVolume(bbox, win_xy, win_z, postIF[n], filepath)

        postsynapticvolumes.append(volume)

    synapticVolumes = {'presynaptic': presynapticvolumes,
                       'postsynaptic': postsynapticvolumes}
                       
    return synapticVolumes

